# Notebook 10: A hyperparameter search

### Goal: Show an example of hyperparameter tuning 

#### Background

If you ever choose to look at any of the ML method documentation, you will find there are alot of switches and nobs you can play with. See this page on ```RandomForestRegressor``` [click](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

#### Step 1 & 2: Import packages and load data for Classification  
In Notebook 5 we only wanted 1 feature, no we want all available inputs (36 total). So all we need to change is the ```features_to_keep``` keyword to include all indices.

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt

df_train = pd.read_csv('/Users/randyjc2/Downloads/hypersearch_all_scores_train_spacetrees.csv',index_col=0)
df_val = pd.read_csv('/Users/randyjc2/Downloads/hypersearch_all_scores_val_spacetrees.csv',index_col=0)


import matplotlib.patheffects as path_effects
pe1 = [path_effects.withStroke(linewidth=2,
                             foreground="k")]
pe2 = [path_effects.withStroke(linewidth=2,
                             foreground="w")]

fig,axes = plt.subplots(2,2,figsize=(5,5))
fig.set_facecolor('w')

axes = axes.ravel()


n_tree = [1,5,10,25,50,100,200,400,800,1000]
# select a divergent colormap
cmap = matplotlib.cm.cividis
bounds = n_tree
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N, extend='both')
scalarMap = matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap)


titles = ['Bias','MAE','RMSE','$R^{2}$']
for j,ax in enumerate(axes):
    for i in np.arange(0,100,10):
        color_choice =scalarMap.to_rgba(n_tree[int(i/10)])
        if j == 2:
            ax.plot(np.arange(1,11,1),np.sqrt(df_train[str(j)][i:i+10]),'--o',color=color_choice,ms=3)
            ax.plot(np.arange(1,11,1),np.sqrt(df_val[str(j)][i:i+10]),'-o',color=color_choice,ms=3)
        else:
            ax.plot(np.arange(1,11,1),df_train[str(j)][i:i+10],'o--',color=color_choice,ms=3)
            ax.plot(np.arange(1,11,1),df_val[str(j)][i:i+10],'o-',color=color_choice,ms=3)

        ax.set_title(titles[j])
        ax.set_xlim([0,12])
        ax.grid('on')

        ax.axvline(8,ls=':',color='k')
        
        if j ==3:
            ax.text(0.075, 0.9, annotate_list[j], transform=ax.transAxes,fontsize=fs4,
                verticalalignment='top', bbox=props)
        else:
            ax.text(0.075, 0.25, annotate_list[j], transform=ax.transAxes,fontsize=fs4,
                verticalalignment='top', bbox=props)
        
        
axes[2].set_xlabel('Tree Depth')
axes[3].set_xlabel('Tree Depth')

ax_cbar = fig.add_axes([1.025, 0.4, 0.015,0.33])
cbar = make_colorbar(ax_cbar,1,1000,cmap)
cbar.set_label('# of trees')
plt.tight_layout()

plt.savefig('/Users/randyjc2/Documents/OU_Mac_Files/OU_Stuff/Research/Papers/WAF_ML_Tutorial_V2/Regression_Hyper_tuning.png',dpi=500,bbox_inches='tight')